In [1]:
from neural_network import NeuralNetwork
import numpy as np
import struct

# read in mnist dataset
def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        array = np.fromstring(f.read(), dtype=np.uint8).reshape(shape)
        return array.reshape(array.shape[0],-1).T
        
def convert_to_multiclass(Y):
    '''
    Convert Y from shape(1 x m) to shape (10 x m)
    '''
    Y_list = []
    for each in Y[0,:]:
        this_Y = np.zeros((10,))
        this_Y[each] = 1
        Y_list.append(this_Y)
    
    multiclass = np.array(Y_list).T
    return multiclass


def show_numbers(data, start_idx, end_idx):
    side = np.int8(np.sqrt(data.shape[0]))
    for each in range(start_idx, end_idx):
        pixels = ((data[:,each].reshape(side,side) * 255).astype(np.uint8))
        plt.imshow(pixels, cmap=plt.cm.binary)
        plt.show()

In [2]:
train_X = read_idx('dataset/train_images_ubyte')
train_Y = read_idx('dataset/train_labels_ubyte')
test_X = read_idx('dataset/test_images_ubyte')
test_Y = read_idx('dataset/test_labels_ubyte')

print("Train shape:\nX: {}\nY: {}".format(train_X.shape,train_Y.shape))
print(train_Y[:,:5])

Train shape:
X: (784, 60000)
Y: (1, 60000)
[[5 0 4 1 9]]


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  # Remove the CWD from sys.path while we load stuff.


In [3]:
multiclass_train_Y = convert_to_multiclass(train_Y)
multiclass_test_Y = convert_to_multiclass(test_Y)

print(multiclass_train_Y[:,:5])

[[0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]]


In [4]:
this = np.array([[0,5,3,10,3],
                 [3,2,1,11,4]])
np.argmax(this, axis=0)

array([1, 0, 0, 1, 1])

In [7]:
layer_dims = [train_X.shape[0],50,10]
nn = NeuralNetwork(layer_dims,init_method='he')
print(nn)

Layer 1:
	W.shape: (50, 784)
	b.shape: (50, 1)
	initialization method: he
	activation method: relu

Layer 2:
	W.shape: (10, 50)
	b.shape: (10, 1)
	initialization method: he
	activation method: sigmoid




In [6]:
costs = nn.train(train_X, multiclass_train_Y, batcoptimizer='adam')

Cost after epoch 0/50: 32.81184520011145
